In [1]:
# Import dependencies
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json
import pandas as pd
from pandas.io import sql
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from consts import *
%matplotlib inline

In [2]:
# Connecting to Postgres instance
engine = create_engine(CREATE_ENGINE_STR)

In [3]:
# Printing info for table names 
print (engine.table_names())

['agg_county_votes', 'education', 'committee_summary_2020', 'donations', 'fec_donor_az', 'health_metrics', 'birth_death_rate', 'postal_codes', 'fec_donor_mi', 'fec_donor_wi', 'fec_committee', 'fec_donor_pa', 'pres_votes_6t', 'unemployment', 'fec_donor_nc', 'fec_donor_fl']


In [4]:
def calculate_election_yr(i):
    return (2000 + i*4)

In [5]:
# Definition to take in the votes data frame. Function takes in the votes dataframe with 20 years of data. 
# This will loop thru every 4 years, runs thru all of the county votes then return it in a list. 
# This will aggregate everything and return a list in a df
def get_votes_intervals(votes_df, state_po):
    votes_states_df = votes_df[votes_df['state_po']==state_po]
    i = ELECTION_STARTING_YR
    
    four_yr_dfs = []
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = votes_states_df[votes_states_df['year']==i]    
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
    
    return four_yr_dfs

In [6]:
# Goes thru each county (string), to pull the election date and calculate votes in the county that are democrat (blue), republic (red) and other. 
def vote_distribution(county, election_df, state, i):        
    county_df = election_df[election_df['county']==county]
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #Other = not democratic AND not republican  
    other_votes = 0
    blue_votes = pd.to_numeric(county_blue_df["candidatevotes"].sum(), errors='coerce')
    red_votes = pd.to_numeric(county_red_df["candidatevotes"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_votes += pd.to_numeric(party_df["candidatevotes"].sum(), errors='coerce')
    
    #Total votes it the sum of blue + red + other
    total_votes = blue_votes + red_votes + other_votes
    
    #Get the respective percentages
    percent_blue = (blue_votes / total_votes)
    percent_red = (red_votes / total_votes)
    percent_other = (other_votes / total_votes)
    
    election_year = calculate_election_yr(i)
    
    #Set the unemployment data points from the county for that election year.
    unemployment_sql = f'SELECT * FROM unemployment WHERE "County" = \'{county}\' AND "Stabr" = \'{state}\''
    unemployment_df = pd.read_sql_query(unemployment_sql,con=engine)
    
    #Initialize 
    urban_pct = 0
    unemployment_rate = 0
    urban_den = 0
    rural_pct = 0
    rural_den = 0
    
    if not unemployment_df.empty:
        #Get POPPCT_URBAN -> urban_pct
        unemployment_col = "Unemployment_rate_" + str(election_year)  
        urban_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_URBAN'].values[0]      
        unemployment_rate = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), unemployment_col].values[0]
        urban_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_URBAN'].values[0]
        rural_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_RURAL'].values[0]
        rural_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_RURAL'].values[0]

    county_tuple = (
        blue_votes,
        red_votes,
        other_votes,
        total_votes,
        percent_blue,
        percent_red,
        percent_other,
        county,
        state,
        election_year,
        urban_pct,
        unemployment_rate,
        urban_den,
        rural_pct,
        rural_den
    )
    return county_tuple

In [7]:
def donor_distribution(election_df, county, state, i):    
    county = county.strip()
        
    county_df = election_df[election_df['county']==county]
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #Other = not democratic AND not republican  
    other_amt = 0
    blue_amt = pd.to_numeric(county_blue_df["transaction_amt"].sum(), errors='coerce')
    red_amt = pd.to_numeric(county_red_df["transaction_amt"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_amt += pd.to_numeric(party_df["transaction_amt"].sum(), errors='coerce')
    
    #Total transaction amount it the sum of blue + red + other
    total_amt = blue_amt + red_amt + other_amt
    
    #Get the respective percentages
    percent_blue = (blue_amt / total_amt)
    percent_red = (red_amt / total_amt)
    percent_other = (other_amt / total_amt)
    
    election_year = calculate_election_yr(i)
    donor_tuple = (
        blue_amt,
        red_amt,
        other_amt,
        total_amt,
        percent_blue,
        percent_red,
        percent_other,
        county,
        state,
        election_year
    )
    return donor_tuple

In [8]:
# Run every election year in the loop, get election df, look for the vote distribution
# Will be run on a single state and will return a dictionary tha tcountains every county in the state as a key. 
# The value (number of votes) is a dictionary of values.
def county_vote_distribution(four_yr_dfs, state):
    #Loop through each election DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        unique_counties = election_df["county"].unique()
        print(f"Running county election year: {calculate_election_yr(i)} num countines: {len(unique_counties)}")
        
        #Loop through each unique county
        for county in unique_counties:
            print(f"Running county: {county}")
            #Get the percent of the vote distribution for that county
            county_tuple = vote_distribution(county, election_df, state, i)            
            county_votes_df = pd.DataFrame([county_tuple], columns=VOTES_COLS)
            #print("county_votes_df")
            #print(county_votes_df.size)
            #print(county_votes_df.head())
            #Write the vote tallies per county to DB
            county_votes_df.to_sql(TABLE_AGG_VOTES, con=engine, if_exists="append")

In [9]:
"""
ACE	Ace Party	
AKI	Alaskan Independence Party	
AIC	American Independent Conservative	
AIP	American Independent Party	
AMP	American Party	
APF	American People's Freedom Party	
AE	Americans Elect	
CIT	Citizens' Party	
CMD	Commandments Party	
CMP	Commonwealth Party of the U.S.	
COM	Communist Party	
CNC	Concerned Citizens Party Of Connecticut	
CRV	Conservative Party	
CON	Constitution Party	
CST	Constitutional	
COU	Country	
DCG	D.C. Statehood Green Party	
DNL	Democratic -Nonpartisan League	
DEM	Democratic Party	
D/C	Democratic/Conservative	
DFL	Democratic-Farmer-Labor	
DGR	Desert Green Party	
FED	Federalist	
FLP	Freedom Labor Party	
FRE	Freedom Party	
GWP	George Wallace Party	
GRT	Grassroots	
GRE	Green Party	
GR	Green-Rainbow	
HRP	Human Rights Party	
IDP	Independence Party	
IND	Independent	
IAP	Independent American Party	
ICD	Independent Conservative Democratic	
IGR	Independent Green	
IP	Independent Party	
IDE	Independent Party of Delaware	
IGD	Industrial Government Party	
JCN	Jewish/Christian National	
JUS	Justice Party	
LRU	La Raza Unida	Also see RUP
LBR	Labor Party	Also see LAB
LFT	Less Federal Taxes	
LBL	Liberal Party	
LIB	Libertarian Party	
LBU	Liberty Union Party	
MTP	Mountain Party	
NDP	National Democratic Party	
NLP	Natural Law Party	
NA	New Alliance	
NJC	New Jersey Conservative Party	
NPP	New Progressive Party	
NPA	No Party Affiliation	
NOP	No Party Preference	Commonly used in CA & WA
NNE	None	
N	Nonpartisan	
NON	Non-Party	
OE	One Earth Party	
OTH	Other	
PG	Pacific Green	
PSL	Party for Socialism and Liberation	
PAF	Peace And Freedom	Also see PFP
PFP	Peace And Freedom Party	Also see PAF
PFD	Peace Freedom Party	
POP	People Over Politics	
PPY	People's Party	
PCH	Personal Choice Party	
PPD	Popular Democratic Party	
PRO	Progressive Party	
NAP	Prohibition Party	
PRI	Puerto Rican Independence Party	
RUP	Raza Unida Party	Also see LRU
REF	Reform Party	
REP	Republican Party	
RES	Resource Party	
RTL	Right To Life	
SEP	Socialist Equality Party	
SLP	Socialist Labor Party	
SUS	Socialist Party	
SOC	Socialist Party U.S.A.	
SWP	Socialist Workers Party	
TX	Taxpayers	
TWR	Taxpayers Without Representation	
TEA	Tea Party	
THD	Theo-Democratic	
LAB	U.S. Labor Party	Also see LBR
USP	U.S. People's Party	
UST	U.S. Taxpayers Party	
UN	Unaffiliated	
UC	United Citizen	
UNI	United Party	
UNK	Unknown	
VET	Veterans Party	
WTP	We the People	
W	Write-In
"""

"\nACE\tAce Party\t\nAKI\tAlaskan Independence Party\t\nAIC\tAmerican Independent Conservative\t\nAIP\tAmerican Independent Party\t\nAMP\tAmerican Party\t\nAPF\tAmerican People's Freedom Party\t\nAE\tAmericans Elect\t\nCIT\tCitizens' Party\t\nCMD\tCommandments Party\t\nCMP\tCommonwealth Party of the U.S.\t\nCOM\tCommunist Party\t\nCNC\tConcerned Citizens Party Of Connecticut\t\nCRV\tConservative Party\t\nCON\tConstitution Party\t\nCST\tConstitutional\t\nCOU\tCountry\t\nDCG\tD.C. Statehood Green Party\t\nDNL\tDemocratic -Nonpartisan League\t\nDEM\tDemocratic Party\t\nD/C\tDemocratic/Conservative\t\nDFL\tDemocratic-Farmer-Labor\t\nDGR\tDesert Green Party\t\nFED\tFederalist\t\nFLP\tFreedom Labor Party\t\nFRE\tFreedom Party\t\nGWP\tGeorge Wallace Party\t\nGRT\tGrassroots\t\nGRE\tGreen Party\t\nGR\tGreen-Rainbow\t\nHRP\tHuman Rights Party\t\nIDP\tIndependence Party\t\nIND\tIndependent\t\nIAP\tIndependent American Party\t\nICD\tIndependent Conservative Democratic\t\nIGR\tIndependent Green\t\

In [10]:
def map_zip_county(unique_zips, state_zips):
    county_dict = {}
    unique_counties = {}
    for zipcode in unique_zips:
        #Filter out on the zip code from the state_zips DF
        county_zip = state_zips[state_zips["zip"] == zipcode]
        #Get the county name from the DF and convert it to lower
        county_name = county_zip["county"].to_string(index=False).strip().lower()
        #Filter out the county string within
        county_name = county_name.replace(" county", "").capitalize()
        
        county_dict[zipcode] = county_name
        if county_name not in unique_counties:
            unique_counties[county_name] = True
            
    return (county_dict, unique_counties.keys())

In [11]:
def one_hot_encode(df):
    # Generate our categorical variable list
    cat_vars = df.dtypes[df.dtypes == "object"].index.tolist()

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(df[cat_vars]))

    # Add the encoded variable names to the DataFrame
    encode_df.columns = enc.get_feature_names(cat_vars)
    
    return encode_df

In [12]:
#Add a new column party to the DF that maps the committee party abbreviation to a major party
def merge_cmtid_party(donor_df):        
    #Get the major party strings to map to 
    party_repub = MAJOR_PARTIES[1]
    party_democrat = MAJOR_PARTIES[0]
    party_other = "other"
    
    #Map the affiliation code to the party affiliation
    cmte_party_map = {
        "REP": party_repub,
        "TEA": party_repub,
        "DNL": party_democrat,
        "DNL": party_democrat,
        "DEM": party_democrat,
        "D/C": party_democrat,
        "DFL": party_democrat,
        "THD": party_democrat,
        "PPD": party_democrat,
        "UNK": party_other
    }
    
    donor_df["party"] = donor_df["cmte_pty_affiliation"].map(cmte_party_map)
    
    return donor_df

In [13]:
#Loop through each of the election year DFs and 
def donation_county_cycle_distribution(four_yr_dfs, state_zips, committee_df, state):
    #Loop through each election year DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        #Without zipcode can't do a county lookup, so drop all null values
        election_df.dropna(subset=["zip"], inplace=True)
        #Get the unique values of zip code in the election DF
        unique_zips = election_df["zip"].unique()
        #Createa a map of zip to county, and a list of all unique counties in that state
        (zip_county_map, unique_counties) = map_zip_county(unique_zips, state_zips)
        #Map the zipcode to the county name per the map function
        election_df["county"] = election_df["zip"].map(zip_county_map)
        
        print(f"Running donor distribution election yr: {calculate_election_yr(i)} num countines: {len(unique_counties)}")

        #Loop through each unique county
        for county in unique_counties:
            print(f"Running county: {county}")
            #Get the donor distribution for that county, state, election year as a tuple
            donor_tuple = donor_distribution(election_df, county, state, i)
            #Create a DF to store the county donor info
            donor_df = pd.DataFrame([donor_tuple], columns = DONOR_COLS)  
            #Write the donation amounts to the DB
            donor_df.to_sql(TABLE_AGG_DONORS, con=engine, if_exists="append")

In [14]:
def str_dt(donor_date_str):
    #01/01/1996 - 12/31/1999
    donor_date = datetime.strptime(donor_date_str, '%Y-%m-%d')
    return donor_date

In [15]:
def get_year_from_date_str(donor_date_str):
    donor_date = str_dt(donor_date_str)
    donor_year = donor_date.year
    return donor_year

In [16]:
def get_donors_intervals(donor_df, state):
    donors_states_df = donor_df[donor_df['state']==state.lower()]
    
    i = ELECTION_STARTING_YR
    prev_year = ELECTION_STARTING_YR - ELECTION_INTERVAL
    ending_yr = ELECTION_ENDING_YR
    
    four_yr_dfs = []
        
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = donors_states_df[(donors_states_df['transaction_dt']>datetime.date(prev_year,1,1)) & (donors_states_df['transaction_dt']<datetime.date(i,3,1))]          
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
        prev_year += ELECTION_INTERVAL
        
    return four_yr_dfs

In [17]:
#Get all donation records for a single state and return it in a dataframe
def donor_state_query(state, engine):
    #Run queries to get all donation records from the states into dfs
    donor_table_name = '"fec_donor_{}"'.format(state.lower())    
    donor_select_sql = 'select * from {}'.format(donor_table_name)
    donor_df = pd.read_sql_query(donor_select_sql,con=engine)
    return donor_df

In [18]:
#Aggregate tables are the output of this script, drop them to start fresh
def drop_agg_tables():
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_DONORS, engine)
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_VOTES, engine)

In [19]:
#Main Loop of the program
def main():
    #Read the various tables into DFs
    health_df = pd.read_sql_query('select * from "health_metrics"',con=engine)
    committee_df = pd.read_sql_query('select * from "fec_committee"',con=engine)
    votes_df = pd.read_sql_query('select * from "pres_votes_6t"',con=engine)
    zips_df = pd.read_sql_query('select * from "postal_codes"',con=engine)
    
    #Lowercase the column
    committee_df['cmte_id'] = committee_df['cmte_id'].str.lower()
    
    #Drop the aggregate tables to do fresh data analysis
    drop_agg_tables()

    #Loop through each state
    for state in SWING_STATES:
        print("State:" + state)
        print("Aggregating Vote and Donation records...")
        #Get the votes related to that state
        votes_intervals_df = get_votes_intervals(votes_df, state)

        #Get the distribution of Red, Blue, and Other votes in a list of dict per election yr e.g. 2000 + 4n
        county_vote_distribution(votes_intervals_df, state)
        
        #DF that has all donation records for a state
        donor_df_orig = donor_state_query(state, engine)
        
        #Add party column to donor data frame
        donor_df = committee_df.merge(donor_df_orig, left_on='cmte_id', right_on='cmt_id')
        donor_df = merge_cmtid_party(donor_df)
        
        #Get a list of DFs per election year per state
        donors_intervals_df = get_donors_intervals(donor_df, state)
        #Filter out the zips DF by the state
        state_zips = zips_df[zips_df["state"] == state]
                
        #Get list of tuples 
        donation_county_cycle_distribution(donors_intervals_df, state_zips, committee_df, state)
        
        print("\n")
    
    print("Swing States Aggregation Done!")

In [20]:
#Run the main loop
main()

State:AZ
Aggregating Vote and Donation records...
Running county election year: 2000 num countines: 15
Running county: Apache
Running county: Cochise
Running county: Coconino
Running county: Gila
Running county: Graham
Running county: Greenlee
Running county: La Paz
Running county: Maricopa
Running county: Mohave
Running county: Navajo
Running county: Pima
Running county: Pinal
Running county: Santa Cruz
Running county: Yavapai
Running county: Yuma
Running county election year: 2004 num countines: 15
Running county: Apache
Running county: Cochise
Running county: Coconino
Running county: Gila
Running county: Graham
Running county: Greenlee
Running county: La Paz
Running county: Maricopa
Running county: Mohave
Running county: Navajo
Running county: Pima
Running county: Pinal
Running county: Santa Cruz
Running county: Yavapai
Running county: Yuma
Running county election year: 2008 num countines: 15
Running county: Apache
Running county: Cochise
Running county: Coconino
Running county: Gil

Running county: Muskegon
Running county: Newaygo
Running county: Oakland
Running county: Oceana
Running county: Ogemaw
Running county: Ontonagon
Running county: Osceola
Running county: Oscoda
Running county: Otsego
Running county: Ottawa
Running county: Presque Isle
Running county: Roscommon
Running county: Saginaw
Running county: St. Clair
Running county: St. Joseph
Running county: Sanilac
Running county: Schoolcraft
Running county: Shiawassee
Running county: Tuscola
Running county: Van Buren
Running county: Washtenaw
Running county: Wayne
Running county: Wexford
Running county election year: 2012 num countines: 83
Running county: Alcona
Running county: Alger
Running county: Allegan
Running county: Alpena
Running county: Antrim
Running county: Arenac
Running county: Baraga
Running county: Barry
Running county: Bay
Running county: Benzie
Running county: Berrien
Running county: Branch
Running county: Calhoun
Running county: Cass
Running county: Charlevoix
Running county: Cheboygan
Runni

Running county: Marion
Running county: Martin
Running county: Miami-Dade
Running county: Monroe
Running county: Nassau
Running county: Okaloosa
Running county: Okeechobee
Running county: Orange
Running county: Osceola
Running county: Palm Beach
Running county: Pasco
Running county: Pinellas
Running county: Polk
Running county: Putnam
Running county: St. Johns
Running county: St. Lucie
Running county: Santa Rosa
Running county: Sarasota
Running county: Seminole
Running county: Sumter
Running county: Suwannee
Running county: Taylor
Running county: Union
Running county: Volusia
Running county: Wakulla
Running county: Walton
Running county: Washington
Running county election year: 2008 num countines: 67
Running county: Alachua
Running county: Baker
Running county: Bay
Running county: Bradford
Running county: Brevard
Running county: Broward
Running county: Calhoun
Running county: Charlotte
Running county: Citrus
Running county: Clay
Running county: Collier
Running county: Columbia
Running c

Running county: Rowan
Running county: Rutherford
Running county: Sampson
Running county: Scotland
Running county: Stanly
Running county: Stokes
Running county: Surry
Running county: Swain
Running county: Transylvania
Running county: Tyrrell
Running county: Union
Running county: Vance
Running county: Wake
Running county: Warren
Running county: Washington
Running county: Watauga
Running county: Wayne
Running county: Wilkes
Running county: Wilson
Running county: Yadkin
Running county: Yancey
Running county election year: 2004 num countines: 100
Running county: Alamance
Running county: Alexander
Running county: Alleghany
Running county: Anson
Running county: Ashe
Running county: Avery
Running county: Beaufort
Running county: Bertie
Running county: Bladen
Running county: Brunswick
Running county: Buncombe
Running county: Burke
Running county: Cabarrus
Running county: Caldwell
Running county: Camden
Running county: Carteret
Running county: Caswell
Running county: Catawba
Running county: Chat

Running county: Camden
Running county: Carteret
Running county: Caswell
Running county: Catawba
Running county: Chatham
Running county: Cherokee
Running county: Chowan
Running county: Clay
Running county: Cleveland
Running county: Columbus
Running county: Craven
Running county: Cumberland
Running county: Currituck
Running county: Dare
Running county: Davidson
Running county: Davie
Running county: Duplin
Running county: Durham
Running county: Edgecombe
Running county: Forsyth
Running county: Franklin
Running county: Gaston
Running county: Gates
Running county: Graham
Running county: Granville
Running county: Greene
Running county: Guilford
Running county: Halifax
Running county: Harnett
Running county: Haywood
Running county: Henderson
Running county: Hertford
Running county: Hoke
Running county: Hyde
Running county: Iredell
Running county: Jackson
Running county: Johnston
Running county: Jones
Running county: Lee
Running county: Lenoir
Running county: Lincoln
Running county: McDowell
R

Running county: Bradford
Running county: Bucks
Running county: Butler
Running county: Cambria
Running county: Cameron
Running county: Carbon
Running county: Centre
Running county: Chester
Running county: Clarion
Running county: Clearfield
Running county: Clinton
Running county: Columbia
Running county: Crawford
Running county: Cumberland
Running county: Dauphin
Running county: Delaware
Running county: Elk
Running county: Erie
Running county: Fayette
Running county: Forest
Running county: Franklin
Running county: Fulton
Running county: Greene
Running county: Huntingdon
Running county: Indiana
Running county: Jefferson
Running county: Juniata
Running county: Lackawanna
Running county: Lancaster
Running county: Lawrence
Running county: Lebanon
Running county: Lehigh
Running county: Luzerne
Running county: Lycoming
Running county: McKean
Running county: Mercer
Running county: Mifflin
Running county: Monroe
Running county: Montgomery
Running county: Montour
Running county: Northampton
Runni

Running county: Kenosha
Running county: Kewaunee
Running county: La Crosse
Running county: Lafayette
Running county: Langlade
Running county: Lincoln
Running county: Manitowoc
Running county: Marathon
Running county: Marinette
Running county: Marquette
Running county: Menominee
Running county: Milwaukee
Running county: Monroe
Running county: Oconto
Running county: Oneida
Running county: Outagamie
Running county: Ozaukee
Running county: Pepin
Running county: Pierce
Running county: Polk
Running county: Portage
Running county: Price
Running county: Racine
Running county: Richland
Running county: Rock
Running county: Rusk
Running county: St. Croix
Running county: Sauk
Running county: Sawyer
Running county: Shawano
Running county: Sheboygan
Running county: Taylor
Running county: Trempealeau
Running county: Vernon
Running county: Vilas
Running county: Walworth
Running county: Washburn
Running county: Washington
Running county: Waukesha
Running county: Waupaca
Running county: Waushara
Running